### Solving Coding Challenge

In [14]:
import os
from typing import List, Tuple, Optional

In [2]:
file_path = 'data/challenge_data.txt'

In [148]:
class Coordinate:
    
    def __init__(self, x:int, y:int):
        self._x = x
        self._y = y
        
    def __str__(self):
        return f'({self._x}, {self._y})'
    
    @property
    def to_tuple(self) -> Tuple[int]:
        return (self._x, self._y)
    
    @property
    def x(self) -> int:
        return self._x
    
    @property
    def y(self) -> int:
        return self._y
    
    @property
    def min(self) -> int:
        return min(self.to_tuple)
    
    @property
    def max(self) -> int:
        return max(self.to_tuple)
    
    @staticmethod
    def from_string(string: str, sep: Optional[str] = ','):
        lst = string.split(sep)
        if len(lst) == 2:
            try:
                x = int(lst[0])
                y = int(lst[1])
                return Coordinate(x, y)
            except ValueError:
                return None
        return None


class CoordinatePair:
    
    def __init__(self, start_coordinate: Coordinate, end_coordinate: Coordinate):
        self._start = start_coordinate
        self._end = end_coordinate
    
    def __str__(self):
        return f'{str(self._start)} -> {str(self._end)}'
    
    def __repr__(self):
        return str(self)
    
    @staticmethod
    def from_string(source: str, coordinate_separator: str='\t', pair_separator: str=','):
        coordinate_split = source.split(coordinate_separator)
        assert len(coordinate_split) == 2, f'{source} would produce more than two coordinates'
        
        coordinate_1 = Coordinate.from_string(coordinate_split[0], pair_separator)
        coordinate_2 = Coordinate.from_string(coordinate_split[1], pair_separator)
        return CoordinatePair(coordinate_1, coordinate_2)
        
    @property
    def is_valid(self) -> bool:
        return self._start.x == self._end.x or self._start.y == self._end.y
    
    @property
    def common_value(self) -> int:
        if self.is_valid:
            if self._start.x == self._end.x:
                return self._start.x
            elif self._start.y == self._end.y:
                return self._end.y
        return None
    
    @property
    def is_ascendent(self) -> bool:
        max_index = self.__get_max_value_coord()
        min_index = self.__get_min_value_coord()
        
        if max_index != None and min_index != None:
            if max_index > min_index:
                return False
            elif max_index < min_index:
                return True
        return None
    
    @property
    def max(self) -> int:
        if self.is_valid:
            start_max = self._start.max if self._start.max != self.common_value else self._start.min
            end_max = self._end.max if self._end.max != self.common_value else self._end.min
            return max(start_max, end_max)
        return None 
    
    @property
    def min(self) -> int:
        if self.is_valid:
            start_min = self._start.min if self._start.min != self.common_value else self._start.max
            end_min = self._end.min if self._end.min != self.common_value else self._end.max
            return min(start_min, end_min)
        return None
    
    @property
    def coordinates() -> Tuple[Coordinate]:
        return (self._start, self._end)
    
    def __get_max_value_coord(self):
        max_value = self.max
        if max_value:
            if max_value == self._start.x or max_value == self._start.y:
                return 0
            elif max_value == self._end.x or max_value == self._end.y:
                return 1
        return None
    
    def __get_min_value_coord(self):
        min_value = self.min
        if min_value:
            if min_value == self._start.x or min_value == self._start.y:
                return 0
            elif min_value == self._end.x or min_value == self._end.y:
                return 1
        return None
    
    def __get_common_value_index(self):
        common_value = self.common_value
        if common_value:
            if common_value == self._start.x and common_value == self._end.x:
                return 0
            elif common_value == self._start.y and common_value == self._end.y:
                return 1
        return None
    
    def generate_intervals(self, increment:int=1):
        increment = abs(increment)
        if self.is_valid:
            common_value = self.common_value
            common_value_index = self.__get_common_value_index()
            intervals = []
            interval_range = None
            if self.is_ascendent:
                interval_range = tuple(range(self.min, self.max + 1, increment))
            elif self.is_ascendent == False:
                interval_range = tuple(range(self.max, self.min - 1, -1 * increment))
            
            if interval_range:
                for i in interval_range:
                    if common_value_index == 0:
                        intervals.append((common_value, i))
                    elif common_value_index == 1:
                        intervals.append((i, common_value))
            return intervals
        return None

class CoordinatePairSummarizer:
    
    def __init__(self, coordinate_pairs: List[CoordinatePair]):
        self.__coordinate_pairs = list(filter(lambda pair: pair.is_valid, coordinate_pairs))
        
    def summarize(self, interval_increment: int=1):
        interval_dict = {}
        for coord_pair in self.__coordinate_pairs:
            intervals = coord_pair.generate_intervals(interval_increment)
            if not intervals:
                continue
            for interval in intervals:
                key = str(interval)
                current_value = interval_dict.get(key, None)
                interval_dict[key] = current_value + 1 if current_value else 1
        return interval_dict
    
    @property
    def summary(self):
        summary = self.summarize()
        filtered = list(filter(lambda x: x >= 2, list(summary.values())))
        return {
            'max_coord_repetition': max(summary.values()),
            'min_coord_repetition': min(summary.values()),
            'total_initial_valid_coords': len(self.__coordinate_pairs),
            'total_repeated_coords': len(filtered)
        }
    
    def print_summary(self):
        summary = self.summary
        print(f'Maximum coordinate repetitions: {summary.get("max_coord_repetition")}')
        print(f'Minimum coordinate repetitions: {summary.get("min_coord_repetition")}')
        print(f'Total initial valid coordinates: {summary.get("total_initial_valid_coords")}')
        print(f'Total coordinates repeated twice or more: {summary.get("total_repeated_coords")}')
    

In [149]:
class TextFileReader:
    
    @staticmethod
    def read_file(path) -> List[str]:
        lines = []
        if os.path.exists(path):
            with open(path) as file:
                lines = list(map(lambda line: line.rstrip(), file.readlines()))
        return lines
    

class CoordinatePairGenerator:
    
    @staticmethod
    def generate_coordinate_pairs(line_list: List[str], coord_separator: str='->', pair_separator: str=',') -> List[CoordinatePair]:
        return [CoordinatePair.from_string(line, coord_separator, pair_separator) for line in line_list]



In [152]:


def main():
    file_path = 'data/challenge_data.txt'
    lines = TextFileReader.read_file(file_path)
    coordinate_pairs = CoordinatePairGenerator.generate_coordinate_pairs(lines)
    summarizer = CoordinatePairSummarizer(coordinate_pairs)
    summarizer.print_summary()

In [153]:
main()

Maximum coordinate repetitions: 4
Minimum coordinate repetitions: 1
Total initial valid coordinates: 324
Total coordinates repeated twice or more: 5631
